In [15]:
# -*- coding: utf-8 -*-
"""
Created on Mon Feb 26 23:20:19 2018

@author: outline by jpmaldonado, edited by Radek Bartyzal
"""

import numpy as np

solution = np.array([1, 1, -0.5])
# the function we want to optimize for sanity check
def f(theta):
  # Here would go the evaluation of the episode
  reward = -np.sum(np.square(solution - theta))
  return reward

def vecf(thetas):
    return np.array([f(theta) for theta in thetas])

def evaluate_and_sort(cands, f):
    cands_and_scores = []
    for cand in cands:
        cands_and_scores.append(np.array([cand,f(cand)]))
        
    cands_and_scores = np.array(cands_and_scores)
    sorted_cands_and_scores = sorted(cands_and_scores,key=lambda x: x[1])[::-1] # sort according to 2nd column and then reverse the array
    return np.array(sorted_cands_and_scores)

def get_top_cands(sorted_cands_and_scores, selection_rate):
    #print("Sorted cands and scores:\n", sorted_cands_and_scores)
    n = (int)(selection_rate * len(sorted_cands_and_scores))
    top_cands = np.copy(sorted_cands_and_scores[0:n,0])
    size = (n,len(top_cands[0]))
    #print("Wanted size =", size)
    #print("Top cands:\n", top_cands)
    top_cands = np.concatenate(top_cands)
    top_cands = np.reshape(top_cands, size)
    #print("Top cands after reshape:\n", top_cands)
    return top_cands

In [5]:
#################################
# STARTER CODE - CEM
#################################

#batch_size = 25 # number of samples per batch = population size
#elite_frac = 0.2 # fraction of samples used as elite set


def cross_entropy(n_iter = 500, batch_size = 25, elite_frac = 0.2):
    dim_theta = 3
    score_best = -999999
    theta_best = [0,0,0]
    theta_mean = np.zeros(dim_theta)
    theta_std = np.ones(dim_theta)

    for it in range(n_iter):
        # Sample parameter vectors 
        samples = np.random.multivariate_normal(theta_mean, np.diag(theta_std), batch_size)   

        # Evaluate candidates = samples
        sorted_cands_and_scores = evaluate_and_sort(samples, f)

        best_cand_score = sorted_cands_and_scores[0,1]
        if best_cand_score > score_best:
            score_best = best_cand_score
            theta_best = sorted_cands_and_scores[0,0]
            print("NEW BEST:", score_best)

        # Get elite parameters
        top_cands = get_top_cands(sorted_cands_and_scores, elite_frac)

        # Update theta_mean, theta_std
        theta_mean = np.mean(top_cands, axis=0)
        theta_std = np.std(top_cands, axis=0)

        if it % 50==0: 
            print("Generation:", it)

    print("Final best score:", score_best)
    print("Final best theta:", theta_best)
    print("Approximating   :", solution)

cross_entropy()

NEW BEST: -0.33632979117506673
Generation: 0
NEW BEST: -0.013721857102162333
NEW BEST: -0.007958863059797839
NEW BEST: -0.0010574525446755175
Generation: 50
Generation: 100
NEW BEST: -0.0001265746178502719
Generation: 150
Generation: 200
Generation: 250
Generation: 300
Generation: 350
Generation: 400
Generation: 450
Final best score: -0.0001265746178502719
Final best theta: [ 0.99060109  1.00439035 -0.50435431]
Approximating   : [ 1.   1.  -0.5]


In [51]:
#################################
# STARTER CODE - NES
#################################
dim_theta = 3
w0 = np.random.randn(3) #initial guess
npop=50
n_iter=2000
sigma=0.1
alpha=0.001

theta_mean = np.zeros(dim_theta)
theta_std = sigma*np.ones(dim_theta)
score_best = -999999
theta_best = [0,0,0]

# init population
population = np.random.multivariate_normal(theta_mean, np.diag(theta_std), npop) 

for it in range(n_iter):
    # Sample vectors from a normal distribution
    noise_samples = np.random.multivariate_normal(theta_mean, np.diag(theta_std), npop) 
    
    # Sample function values by evaluating on the population
    fitness = vecf(population)
    
    best_cand_score = np.max(fitness)
    best_cand = population[np.argmax(fitness)]
    if best_cand_score > score_best:
        score_best = best_cand_score
        theta_best = best_cand
        #print("NEW BEST:", score_best)
    
    # Optional: standardize (substract mean and divide by std)
    
    for dim in range(dim_theta):
        # prepare the noise in 1 dimension
        noise_in_1D = np.zeros(shape=(npop, dim_theta), dtype=float)
        noise_in_1D[:,dim] = noise_samples[:,dim]
        fitness_with_1D_noise = vecf(np.copy(population) + noise_in_1D)

        # estimate partial derivative of fitness function with respect to dimension dim
        partial_derivative = (fitness_with_1D_noise - fitness) / noise_samples[:,dim]

        # gradient ascent - we are trying to maximize fitness
        population[:,dim] += alpha * partial_derivative
    
    if it % 100==0: 
        print("Generation:", it, "\tbest:", best_cand_score, "\tparams:", best_cand)

print("\nFinal best score:", score_best)
print("Final best theta:", theta_best)
print("Approximating   :", solution)
        

Generation: 0 	best: -0.6747145637893706 	params: [0.57602289 0.53933759 0.029385  ]
Generation: 100 	best: -0.45816902020267314 	params: [ 0.65582843  0.61895084 -0.06107872]
Generation: 200 	best: -0.3086533562529282 	params: [ 0.72313317  0.68254335 -0.13921401]
Generation: 300 	best: -0.20567839941257782 	params: [ 0.77541748  0.73974459 -0.20565604]
Generation: 400 	best: -0.139857784903808 	params: [ 0.81650884  0.78388351 -0.25713379]
Generation: 500 	best: -0.09461430287247471 	params: [ 0.84677078  0.821058   -0.3033507 ]
Generation: 600 	best: -0.06519954090079196 	params: [ 0.8696924   0.85073544 -0.33988047]
Generation: 700 	best: -0.04378220368809223 	params: [ 0.89482979  0.88207036 -0.36340235]
Generation: 800 	best: -0.03063620514202138 	params: [ 0.9112988   0.90058975 -0.38666583]
Generation: 900 	best: -0.020512473599346576 	params: [ 0.92961893  0.91742148 -0.40637563]
Generation: 1000 	best: -0.013262628375962534 	params: [ 0.94784565  0.93576634 -0.42073981]
Gener

In [43]:
population = np.random.multivariate_normal(theta_mean, np.diag(theta_std), npop) 
noise_samples = np.random.multivariate_normal(theta_mean, np.diag(theta_std), npop) 
# Sample function values by evaluating on the population
fitness = vecf(population)

print("population: \n", population)
print("noise_samples: \n", noise_samples)
print("fitness: \n", fitness)

for dim in range(dim_theta):
    # prepare the noise in 1 dimension
    noise_in_1D = np.zeros(shape=(npop, dim_theta), dtype=float)
    noise_in_1D[:,dim] = noise_samples[:,dim]
    fitness_with_1D_noise = vecf(np.copy(population) + noise_in_1D)
    
    # estimate partial derivative of fitness function with respect to dimension dim
    partial_derivative = (fitness_with_1D_noise - fitness) / noise_samples[:,dim]
    
    # gradient ascent - we are trying to maximize fitness
    population[:,dim] += alpha * partial_derivative
    
    print("noise_in_1D: \n", noise_in_1D)
    print("fitness_with_1D_noise: \n", fitness_with_1D_noise)
    print("partial_derivative: \n", partial_derivative)
    print("population: \n", population)


population: 
 [[-0.09154397  0.3225386   0.09762542]
 [-0.14630094  0.11865671 -0.30702714]
 [ 0.37146079  0.01033727 -0.45311287]
 [-0.47445231  0.28724907  0.12745486]
 [ 0.39448272  0.18978862 -0.47631313]]
noise_samples: 
 [[ 0.07716516  0.36083847  0.23051791]
 [-0.33943915  0.19287557 -0.57669463]
 [ 0.36466156  0.15554926  0.36215658]
 [-0.34388195 -0.09569706  0.40111661]
 [-0.31678469  0.23596642 -0.08131641]]
fitness: 
 [-2.00757833 -2.12801035 -1.37669226 -3.07572311 -1.02365472]
noise_in_1D: 
 [[ 0.07716516  0.          0.        ]
 [-0.33943915  0.          0.        ]
 [ 0.36466156  0.          0.        ]
 [-0.34388195  0.          0.        ]
 [-0.31678469  0.          0.        ]]
fitness_with_1D_noise: 
 [-1.84507446 -3.02142812 -1.05126214 -4.20805296 -1.50764447]
partial_derivative: 
 [2.10592278 2.63204103 0.89241686 3.29278657 1.52781925]
population: 
 [[-0.08943805  0.3225386   0.09762542]
 [-0.1436689   0.11865671 -0.30702714]
 [ 0.37235321  0.01033727 -0.453112

In [34]:
zeroes = np.zeros(shape=(npop, dim_theta), dtype=float)
zeroes[:,0] = noise_samples[:,0]
zeroes

array([[-0.03127664,  0.        ,  0.        ],
       [ 0.06501752,  0.        ,  0.        ],
       [ 0.08605286,  0.        ,  0.        ],
       [-0.01112053,  0.        ,  0.        ],
       [ 0.11640418,  0.        ,  0.        ]])